# B. Команда аналитиков (задача C/29448) (диагностика технических навыков - аналитика)
* Ограничение времени	1 секунда
* Ограничение памяти	256Mb
* Ввод	стандартный ввод или input.txt
* Вывод	стандартный вывод или output.txt

## Условие:
* Аркадий — менеджер команды аналитиков из $k$ человек (*мы считаем всех членов команды равнозначными*). 
* У Аркадия есть бэклог из $n$ задач, $i$-я задача требует $t_i$ дней работы *любого* из членов команды . 
* Над каждой задачей от начала до конца должен работать кто-то один — передавать задачи в процессе выполнения неудобно. 
* Для каждой задачи известен дедлайн — $d_i$ рабочих дней.

## Найти:
Помогите Аркадию определить, **успеет ли его команда выполнить все задачи в срок**.

## Формат ввода:
* В первой строке заданы два целых положительных числа — $n$ и $k$ ($1 \le n$, $k \le 15$). 
* В каждой из следующих $n$ строк заданы два целых положительных числа — $t_i$ и $d_i$ ($1 \le t_i \le d_i \le 109$).

## Формат вывода:
* Выведите `NO`, если нельзя выполнить все задачи в срок.

* Иначе в первой строке выведите `YES`. 
* Далее выведите $k$ строк, причем в $i$-й строке будет описание того, какие задачи выполняет $i$-й сотрудник: 
 * сначала выведите одно целое неотрицательное число $T_i$ — количество задач, которое будет выполнять $i$-й сотрудник, 
 * а зачем выведите $T_i$ чисел — номера задач, которые будет выполнять i-й сотрудник. 
 * (Выводите номера задач в том порядке, в котором их должен выполнять сотрудник. Задачи нумеруются в порядке перечисления во входных данных.)

Если существует несколько правильных ответов, вы можете вывести любой.

In [1]:
import pandas as pd
import numpy as np 
#--------------------------------
# arrange the input of given data

tmp_str = input()
n = int(tmp_str.split()[0])
k = int(tmp_str.split()[1])

if (1 <= n <= 15) and (1 <= k <= 15):
    nk_ok = True
else:
    nk_ok = False

if nk_ok:
    tasks = []
    for i in range(n):
        tmp_str = input()
        no = i+1                     # keep initial tasks numbering as task_id
        t = int(tmp_str.split()[0])
        d = int(tmp_str.split()[1])
        f = False                    # assigned status flag
        tasks.append((no,t,d,f))

        tasks_df = pd.DataFrame( 
            tasks,
            columns = ['task_id', 'time_todo', 'deadline', 'assigned']
        )

    if ( (tasks_df['time_todo'] < 1).sum() + (tasks_df['time_todo'] > tasks_df['deadline']).sum()
         + (tasks_df['deadline'] > 109).sum() ) == 0:
        input_ok = True
    else:
        input_ok = False

# IDEA of solving:
# the problem looks like scrum sprint burndown chart for the sprint tasks - turned clockwise 90 degree:
# we shall play a kind of tetris with the goal of pack vertical bars representing all n-tasks in k-columns-can
# where each level of can corresponds to one day of the scrum-sprint of total 109 days debth 
# (actually too long for scrum - so it is more like scrumbut, but for the solution logic it has no difference)

# then how we shall select our tetris bars for packing:
# we start from the last day (bottum of the tetris can) and go backwards in time
# we put first (actually last) the tasks with latest deadlines, if few with the same - then the longest among them
# hard to say how be shure that this tactics failure would really mean NO (not possible!) 
# but if finally we pass the tests with this approach, then our packing tactict was good enough
# (we were not asked for a theorem deduction after all !)   

#----------------------------------------------------------
# sorting tasks_df in the preferences order described above
if nk_ok:
    if input_ok:
        tasks_df.sort_values( by = ['deadline','time_todo'], ascending = [False,False], inplace = True)
        tasks_df.reset_index( drop = True, inplace = True)

#------------------------------------------------------------------
# creating tetris-can as a dataframe table - each day shall be filled either with zero or with task_id
# each column means one guy from the team - so we shall have his assignments easily when finished

        last_day = 109

        tetris_df = pd.DataFrame(
            np.zeros((last_day+1,k), dtype=int),      # we add one mode day - to be shure that value 0 is in each column
            index = [i for i in range(1,last_day+2)],
            columns = [i for i in range(1,k+1)]
        )
        tetris_failed = False
    else:
        tetris_failed = True
else:
    tetris_failed = True
    
#----------------------------------------------------------------------------
# and now: run daily scrum routine for available resources allocation to the most urgent tasks

for day in range(last_day,0,-1):
    if tetris_failed:
        break
    if tasks_df['assigned'].sum() == n:
        break
    
    # see the tasks with not expired deadlines
    for tidx in range(n):
        if tetris_failed:
            break
        guy_for_task_found = False
        if (not tasks_df.loc[tidx,'assigned']) and (tasks_df.loc[tidx,'deadline'] >= day):
            
            # for each found task see the first (any) available guy
            for guy in range(1,k+1):
                if guy_for_task_found:
                    break
                if tetris_df.loc[day,guy] == 0:
                    
                    # put to the found guy the tetris bar (corresponding to the task)
                    start_day = day - tasks_df.loc[tidx,'time_todo'] + 1
                    if start_day < 1:  # the task is too long but we have low time to match the deadline
                        print('NO')
                        tetris_failed = True
                        break
                    for taskday in range(day, start_day-1, -1):
                        tetris_df.loc[taskday,guy] = tasks_df.loc[tidx,'task_id']
                        guy_for_task_found = True
                        tasks_df.loc[tidx,'assigned'] = True

# see if all tasks are in tetris can in the end
if (not tetris_failed) and (tasks_df['assigned'].sum() < n):
    print('NO')
elif not tetris_failed:
    print('YES')
    for i in range(1,k+1):
        Ti = tetris_df[i].nunique() - 1  # as we are sure that the value 0 is available in each column! 
        print(Ti, end = '' )
        for j in list(tetris_df[i].unique()):
            if j != 0:
                print( j, end = '' )
        print()


1 2
1 2
YES
11
0
